In [8]:
import nltk
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold

In [14]:
train_init = pd.read_csv("train.csv")
test_init = pd.read_csv("test.csv")

In [10]:
train_init.head()

,post.id,user.id,gender,topic,sign,date,text,age
0,1,11869,male,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",15
1,2,11869,male,Student,Leo,"13,May,2004",These are the team members: Drewe...,15
2,3,11869,male,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,15
3,4,11869,male,Student,Leo,"12,May,2004",testing!!! testing!!!,15
4,5,16332,male,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,33


In [15]:
test_init.head()

,post.id,user.id,gender,topic,sign,date,text
0,96,4876,female,indUnk,Capricorn,"08,August,2004","urlLink im new to this, ..."
1,97,4876,female,indUnk,Capricorn,"08,August,2004",Election time has rolled aro...
2,108,12227,female,Student,Aries,"31,July,2004","Met Lim this morning, then we..."
3,109,12227,female,Student,Aries,"30,July,2004",Such a boring day. I have no ...
4,110,12227,female,Student,Aries,"29,July,2004","Nuts. As usual, Theo and I pl..."


In [11]:
kf = KFold(n_splits = 5)

In [49]:
train_text = train_init[["user.id", "text", "age"]]
train_text.head()
train_text.shape

(442961, 3)

In [84]:
count_vect = CountVectorizer()
tfidf_transformer = TfidfTransformer()


for train_index, test_index in kf.split(train_text):
    train = np.array(train_text)[train_index, :]
    train = pd.DataFrame(train, columns = ['user.id', 'text', 'age'])

    X_train_counts = count_vect.fit_transform(train.text)
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    X_train_counts.shape
    X_train_tfidf.shape
    
    clf = MultinomialNB().fit(X_train_tfidf, train.age)
    
    test_text = test_init[["user.id", "text"]]
    
    X_new_counts = count_vect.transform(test_text.text)
    
    X_new_tfidf = tfidf_transformer.transform(X_new_counts)
    
    predicted = clf.predict(X_new_tfidf)
    
    cols = ["user.id", "age"]
    test_output = pd.DataFrame(columns = cols)
    test_output[["user.id"]] = test_init[["user.id"]]
    test_output[["age"]] = predicted
    
    test_output.head()

ValueError: Unknown label type: (array([17, 17, 17, ..., 34, 34, 34], dtype=object),)

(442961, 547674)

,user.id,age
0,4876,24
1,4876,24
2,12227,17
3,12227,16
4,12227,17


In [20]:
# train.to_csv("train_FRE.csv", encoding = 'utf-8', index = False)
test_output.to_csv("test_sklearn_output.csv", encoding = 'utf-8', index = False)